## Creating Graph

In [2]:
class Vertex:
    def __init__(self, name) -> None:
        self._id = name
        self._weight = None
        self._adjacent = {}
#------ setting attributes ------#
    def set_id(self, name):
        self._id = name
    def set_weight(self, weight):
        self._weight = weight
    def add_adjacent(self, other_vertex, edge_weight=None):
        self._adjacent[other_vertex._id] = (other_vertex, edge_weight)
#------  getting attributes ------#
    def get_adjacent_ids(self):
        res = {key:value[1] for key, value in self._adjacent.items()}
        return res

In [3]:
class Graph:
    def __init__(self) -> None:
        self.vertices = {}
#------ setting attributes ------#
    def add_vertex(self, name):
        vertex = Vertex(name)
        self.vertices[name] = vertex
        return vertex
    def add_undirected_edge(self, vertex_a, vertex_b, edge_weight=None):
        vertex_a.add_adjacent(other_vertex = vertex_b, edge_weight=edge_weight)
        vertex_b.add_adjacent(other_vertex = vertex_a, edge_weight=edge_weight)
    def add_directed_edge(self, start, end, edge_weight=None):
        start.add_adjacent(other_vertex = end, edge_weight = edge_weight)
#------ getting attributes ------#
    def get_vertex(self, name):
        return self.vertices[name]

## Depth First Search with Stacks

In [4]:
# DFS return True or False only  
# Every time check the first element in the stack, if it is not the target;
# iterate the adjacent vertices and put them in the stack; 
# repeat above steps, avoid visited vertices.
# Keep it going until there is no way and than back to last intersection. 

class Graph_DFS_basic(Graph):
    def get_dfs_basic(self, start_id, stop_id):
        stack = list()
        visited = set()
        
        stack.append(start_id)
        while stack:
            current = stack.pop()
            visited.add(current)
            if current == stop_id:
                return True
            
            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited:
                    stack.append(id)
        return False

In [5]:
g = Graph_DFS_basic()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb)
g.add_undirected_edge(vb,vc)
g.add_undirected_edge(vb,vd)
g.add_undirected_edge(vc,vd)
g.add_undirected_edge(vb,ve)
g.add_undirected_edge(vd,ve)
g.get_dfs_basic('a', 'e')

True

In [6]:
# DFS return Path weights and Path route  
# The attribute weight for each vertex represents the sum of last vertex's weight and the edge weight.
# Using dictionary to record each vertex's last vertex, and iterating the dictionary from end to start find the path.

class Graph_DFS_weights(Graph):
    def get_dfs_route(self, start_id, stop_id):
        stack = list()
        visited = set()
        predecessor = {start_id:None}
        
        self.get_vertex(start_id).set_weight(0)
        stack.append(start_id)
        while stack:
            current = stack.pop()
            visited.add(current)
            if current == stop_id:
                path = []
                while current is not None:
                    path.insert(0, current)
                    current = predecessor[current]
                return True, self.get_vertex(stop_id)._weight, path
            
            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited:
                    vertex[0].set_weight(self.get_vertex(current)._weight + vertex[1])
                    stack.append(id)
                    predecessor[id] = current
        
        return False

In [7]:
g = Graph_DFS_weights()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb, 1)
g.add_undirected_edge(vb,vc, 2)
g.add_undirected_edge(vb,vd, 1)
g.add_undirected_edge(vc,vd, 4)
g.add_undirected_edge(vb,ve, 3)
g.add_undirected_edge(vd,ve, 5)
g.get_dfs_route('a', 'd')

(True, 9, ['a', 'b', 'e', 'd'])

## Breadth First Search Algorithm  
The property of queue is FIFO (first in first out).  
So, BFS is different from DFS, it using FIFO to go through all vertices on the same level and then move to next level vertices.

In [8]:
# Basic BFS just return True or False

from collections import deque

class Graph_BFS_basic(Graph):
    def get_bfs_basic(self, start, goal):
        visited = set()
        queue = deque()

        visited.add(start)
        queue.append(start)
        while queue:
            current = queue.popleft()
            visited.add(current)
            if current == goal:
                return True
            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited:
                    queue.append(id)
                    
        return False

In [9]:
g = Graph_BFS_basic()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb, 1)
g.add_undirected_edge(va,vc, 2)
g.add_undirected_edge(vb,vd, 1)
g.add_undirected_edge(vb,ve, 4)

g.get_bfs_basic('a', 'c')

True

**Note**: 
some times the graph if not sample tree structure but a cyclic structure.  
Although BFS do it well but the return path would be incorrect.  
Pay attention on the existed elements in queue.

In [10]:
# BFS with weight and path

from collections import deque

class Graph_BFS_weight(Graph):
    def get_bfs_weight(self, start, goal):
        visited = set()
        queue = deque()
        predecessor = {start:None}
        self.get_vertex(start).set_weight(0)

        visited.add(start)
        queue.append(start)
        while queue:
            current = queue.popleft()
            visited.add(current)
            if current == goal:
                path = []
                while current is not None:
                    path.insert(0, current)
                    current = predecessor[current]

                return True, path, self.get_vertex(goal)._weight

            for id, vertex in self.get_vertex(current)._adjacent.items():
                if id not in visited and id not in queue:
                    queue.append(id)
                    vertex[0].set_weight(vertex[1]+self.get_vertex(current)._weight)
                    predecessor[id] = current
                    
        return False

In [11]:
g = Graph_BFS_weight()
va = g.add_vertex('a')
vb = g.add_vertex('b')
vc = g.add_vertex('c')
vd = g.add_vertex('d')
ve = g.add_vertex('e')
vf = g.add_vertex('f')
g.add_undirected_edge(va,vb, 1)
g.add_undirected_edge(vb,vc, 3)
g.add_undirected_edge(vb,vd, 2)
g.add_undirected_edge(vb,ve, 1)
g.add_undirected_edge(vc,ve, 4)
g.add_undirected_edge(vc,vd, 1)
g.add_undirected_edge(ve,vf, 3)
g.add_undirected_edge(vd,va, 2)
g.add_undirected_edge(vd,ve, 2)

g.get_bfs_weight('a', 'e')

(True, ['a', 'b', 'e'], 2)

## Dijkstra's Algorithm (Shortest Path First)

In [33]:
import math
import heapq

class DijkstraVertex():
    def __init__(self, name):
        self._id = name
        self._adjacent = dict()
        self._distance = math.inf
        self._visited = False # Mark all nodes unvisited  
        self._previous = None # Predecessor
    def add_neighbour(self, neighbour, weight = 0):
        self._adjacent[neighbour] = weight
    def get_adjacent(self):
        return self._adjacent  
    def get_connections(self):
        return self._adjacent.keys()  
    def get_id(self):
        return self._id
    def get_weight(self, neighbour):
        return self._adjacent[neighbour]
    def set_distance(self, dist):
        self._distance = dist
    def get_distance(self):
        return self._distance
    def set_previous(self, prev):
        self._previous = prev
    def get_previous(self):
        return self._previous
    def set_visited(self):
        self._visited = True
    def is_visited(self):
        return self._visited
    def __str__(self):
        return str(self._id) + ' adjacent: ' + str([x.id for x in self._adjacent])
    def __lt__(self, other):
        return self._distance < other.get_distance()

#------ Dijkstra's vertex ------#
# In Shortest Path First, some new attributes are quite essential
# distance, represents that cost from last vertex to the current one.
# visited, indicates the current vertex has been visited or not.
# previous, the last vertex that come to the current one, and current's distance is sum of previous and edge weight.

class DijkstraGraph:
    def __init__(self):
        self._vertices = dict()
    def add_vertex(self, name):
        new_vertex = DijkstraVertex(name)
        self._vertices[name] = new_vertex
        return new_vertex
    def get_vertex(self, n):
        if n in self._vertices:
            return self._vertices[n]
        else:
            return None
    def get_vertices(self):
        return list(self._vertices.values())
    def add_edge(self, frm, to, cost = 0):
        if frm not in self._vertices:
            self.add_vertex(frm)
        if to not in self._vertices:
            self.add_vertex(to)
        self._vertices[frm].add_neighbour(self._vertices[to], cost)
        self._vertices[to].add_neighbour(self._vertices[frm], cost)
    def dijkstra_spf(self, start):
        start.set_distance(0)

        unvisited_queue = list(self._vertices.values())
        heapq.heapify(unvisited_queue)

        while unvisited_queue:
            current = heapq.heappop(unvisited_queue)
            current.set_visited()

            for next in current._adjacent:
                if next.is_visited():
                    continue
                new_dist = current.get_distance() + current.get_weight(next)
                if new_dist < next.get_distance():
                    next.set_distance(new_dist)
                    next.set_previous(current)
                else:
                    pass
            
            while unvisited_queue:
                heapq.heappop(unvisited_queue)
            unvisited_queue = [v for v in list(self._vertices.values()) if not v.is_visited()]
            heapq.heapify(unvisited_queue)

    def get_shortest_path(self, target, path):
        if len(path) == 0:
            path.append(target.get_id())
        previous = target.get_previous()
        if previous:
            path.append(previous.get_id())
            self.get_shortest_path(previous, path)
        else:
            path.reverse()

#------ Dijkstra's Graph ------#
# The key function is dijkstra_spf which will set distance for every vertices from the start.
# After the work above finished, get_shortest_path func can get shortest path and weight from start above to everyone others.


In [34]:
edges = [
    ('a', 'b', 7),('a', 'f', 14),('a', 'c', 9),('c', 'f', 2),('c', 'b', 10),
    ('e', 'f', 9),('e', 'd', 6),('d', 'c', 11),('d', 'b', 15)
]
ids = ['a', 'b', 'c', 'd', 'e', 'f']
source_id = ids[0]

graph = DijkstraGraph()
for v1,v2,weight in edges:
    graph.add_edge(v1, v2, weight)

In [35]:
graph.dijkstra_spf(graph.get_vertex(source_id))
for v in ids[1:]:
    target = graph.get_vertex(v)
    path = []
    g.get_shortest_path(target, path)
    print(f'The shortest path from {source_id} to {target.get_id()} is: {path}, distance {target.get_distance()}')

The shortest path from a to b is: ['a', 'b'], distance 7
The shortest path from a to c is: ['a', 'c'], distance 9
The shortest path from a to d is: ['a', 'c', 'd'], distance 20
The shortest path from a to e is: ['a', 'c', 'f', 'e'], distance 20
The shortest path from a to f is: ['a', 'c', 'f'], distance 11
